In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    modeling_outputs,
    TrainingArguments,
    Trainer
)
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, log_loss
from peft import get_peft_model, LoraConfig
from torch.nn import functional as F

# Step 5.

In [2]:
MODEL_NAME = "microsoft/deberta-v3-small"
MAX_LENGTH = 512
BATCH_SIZE = 32
LEARNING_RATE = 8e-5
EPOCHS = 3

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device.upper()}")
print(f"Using model: {MODEL_NAME}")

try:
    train_df = pd.read_csv("./data/train.csv")
    test_df = pd.read_csv("./data/test.csv")
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")

def create_target(row):
    if row['winner_model_a'] == 1:
        return 0  # Class 0: A wins
    if row['winner_model_b'] == 1:
        return 1  # Class 1: B wins
    if row['winner_tie'] == 1:
        return 2  # Class 2: Tie
    return -1

train_df['label'] = train_df.apply(create_target, axis=1)


train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

print(f"학습 데이터: {len(train_dataset)}, 테스트 데이터: {len(test_dataset)}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    sep = tokenizer.sep_token
    texts_a = [
        f"prompt: {p} {sep} response A: {a}"
        for p, a in zip(examples['prompt'], examples['response_a'])
    ]
    texts_b = [
        f"prompt: {p} {sep} response B: {b}"
        for p, b in zip(examples['prompt'], examples['response_b'])
    ]

    tokenized_a = tokenizer(
        texts_a,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    
    tokenized_b = tokenizer(
        texts_b,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )

    tokenized_inputs = {
        'input_ids_a': tokenized_a['input_ids'],
        'attention_mask_a': tokenized_a['attention_mask'],
        'input_ids_b': tokenized_b['input_ids'],
        'attention_mask_b': tokenized_b['attention_mask'],
    }
    if 'label' in examples:
        tokenized_inputs["labels"] = examples["label"]

    return tokenized_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Using device: CUDA
Using model: microsoft/deberta-v3-small
학습 데이터: 57477, 테스트 데이터: 3


/home/ubuntu/projects/mlp/venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/57477 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [3]:
base_model = AutoModel.from_pretrained(
    MODEL_NAME,
    device_map=device
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_proj", "value_proj"],
    lora_dropout=0.1,
    bias="none",
)

peft_base_model = get_peft_model(base_model, lora_config)
peft_base_model.print_trainable_parameters()

class DeBERTaClassifier(torch.nn.Module):
    def __init__(self, peft_model, num_labels=3):
        super().__init__()
        self.peft_model = peft_model
        hidden_size = self.peft_model.config.hidden_size 
        self.num_labels = num_labels
        self.cross_attn = torch.nn.ModuleList([
            torch.nn.MultiheadAttention(
                embed_dim=hidden_size,
                num_heads=8,
                dropout=0.1,
                batch_first=True
            ) for _ in range(6)
        ])
        
        self.classifier_head = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(hidden_size * 2, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(hidden_size, self.num_labels)
        )

    def forward(self, 
                input_ids_a=None, attention_mask_a=None, 
                input_ids_b=None, attention_mask_b=None, 
                labels=None):
        
        tokens_a = self.peft_model(
            input_ids=input_ids_a,
            attention_mask=attention_mask_a
        ).last_hidden_state
        
        tokens_b = self.peft_model(
            input_ids=input_ids_b,
            attention_mask=attention_mask_b
        ).last_hidden_state
        
        for cross_attn_layer in self.cross_attn:
            attn_output_a, _ = cross_attn_layer(
                query=tokens_a,
                key=tokens_b,
                value=tokens_b,
                key_padding_mask=(attention_mask_b == 0)
            )

            attn_output_b, _ = cross_attn_layer(
                query=tokens_b,
                key=tokens_a,
                value=tokens_a,
                key_padding_mask=(attention_mask_a == 0)
            )
            tokens_a = attn_output_a
            tokens_b = attn_output_b
        
        pooled_output_a = tokens_a[:, 0]
        pooled_output_b = tokens_b[:, 0]
        
        combined_output = torch.cat((pooled_output_a, pooled_output_b), dim=1)
        
        logits = self.classifier_head(combined_output)
        
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1)) 
        
        return modeling_outputs.SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=None,
            attentions=None,
        )

model = DeBERTaClassifier(peft_base_model, num_labels=3).to(device)

class DualEncoderDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        
        features_a = []
        features_b = []
        labels = []

        for feature in features:
            features_a.append({
                'input_ids': feature['input_ids_a'],
                'attention_mask': feature['attention_mask_a']
            })
            features_b.append({
                'input_ids': feature['input_ids_b'],
                'attention_mask': feature['attention_mask_b']
            })
            if 'labels' in feature:
                labels.append(feature['labels'])

        batch_a = self.tokenizer.pad(
            features_a,
            padding=True,
            return_tensors="pt",
        )
        
        batch_b = self.tokenizer.pad(
            features_b,
            padding=True,
            return_tensors="pt",
        )

        batch = {
            'input_ids_a': batch_a['input_ids'],
            'attention_mask_a': batch_a['attention_mask'],
            'input_ids_b': batch_b['input_ids'],
            'attention_mask_b': batch_b['attention_mask'],
        }

        if labels:
            batch['labels'] = torch.tensor(labels, dtype=torch.long)
            
        return batch
    
dual_collator = DualEncoderDataCollator(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = F.softmax(torch.tensor(logits), dim=-1).numpy()
    epsilon = 1e-15
    probs = np.clip(probs, epsilon, 1 - epsilon)
    logloss = log_loss(labels, probs)
    return {"loss": logloss}


print("Trainer 설정 중...")
training_args = TrainingArguments(
    output_dir="./final_2mlp",
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    warmup_steps=100,
    weight_decay=0.01,

    logging_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    fp16=True if device == 'cuda' else False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    compute_metrics=compute_metrics,
    data_collator=dual_collator

)

print("--- 학습 시작 ---")
trainer.train(resume_from_checkpoint=True)
print("--- 학습 완료 ---")


# --- 6. 테스트 데이터 예측 및 제출 ---
print("테스트 데이터 예측 중...")
predictions = trainer.predict(tokenized_test)

# Logits -> Probabilities
test_logits = predictions.predictions
test_probs = F.softmax(torch.tensor(test_logits), dim=-1).numpy()

print(f"테스트 예측 확률 Shape: {test_probs.shape}")

# 제출 파일 생성
submission_df = pd.DataFrame({'id': test_df['id']})
submission_df['winner_model_a'] = test_probs[:, 0] # Class 0
submission_df['winner_model_b'] = test_probs[:, 1] # Class 1
submission_df['winner_tie']     = test_probs[:, 2] # Class 2

submission_df.to_csv("submission.csv", index=False)
print("제출 파일 'submission_step5_deberta_lora.csv' 생성이 완료되었습니다.")
print(submission_df.head())

trainable params: 294,912 || all params: 141,599,232 || trainable%: 0.2083
Trainer 설정 중...
--- 학습 시작 ---


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
4000,1.010900
5000,1.006100


--- 학습 완료 ---
테스트 데이터 예측 중...


테스트 예측 확률 Shape: (3, 3)
제출 파일 'submission_step5_deberta_lora.csv' 생성이 완료되었습니다.
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.265886        0.274929    0.459185
1   211333        0.299775        0.369991    0.330234
2  1233961        0.225737        0.489218    0.285045
